In [1]:
# You must run Jupyter lab as admin, otherwise Cybos object connection will fail.
import datetime
import time
import logging
from logging.handlers import TimedRotatingFileHandler

import win32com.client
import pandas as pd

In [2]:
# 로그 파일 핸들러
fh_log = TimedRotatingFileHandler("logs/log", when="midnight", encoding="utf-8", backupCount=120)
fh_log.setLevel(logging.INFO)
#fh_log.setLevel(logging.DEBUG)

# 콘솔 핸들러
sh = logging.StreamHandler()
#sh.setLevel(logging.DEBUG)
sh.setLevel(logging.INFO)

# 로깅 포멧 설정
formatter = logging.Formatter("[%(asctime)s][%(levelname)s] %(message)s")
fh_log.setFormatter(formatter)
sh.setFormatter(formatter)

# 로거 생성
logger = logging.getLogger("creon")
logger.setLevel(logging.INFO)
# logger.setLevel(logging.DEBUG)
logger.addHandler(fh_log)
logger.addHandler(sh)

In [11]:
class Creon:
    def __init__(self):
        self.obj_CpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
        self.obj_CpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
        self.obj_StockChart = win32com.client.Dispatch("CpSysDib.StockChart")

    def creon_7400_주식차트조회(self, code, date_from, date_to):
        """
        http://money2.creontrade.com/e5/mboard/ptype_basic/HTS_Plus_Helper/DW_Basic_Read_Page.aspx?boardseq=284&seq=102&page=1&searchString=StockChart&p=8841&v=8643&m=9505
        date_from and date_to seems for for both str and int types.
        e.g., 20190830 or "20190830", they work the same.
        :return:
        """
        b_connected = self.obj_CpCybos.IsConnect
        if b_connected == 0:
            logger.critical("연결 실패")
            return None

        list_field_key = [0, 1, 2, 3, 4, 5, 8]
        list_field_name = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']
        dict_chart = {name: [] for name in list_field_name}

        while True:
            self.obj_StockChart.SetInputValue(0, f'A{code}')
            self.obj_StockChart.SetInputValue(1, ord('1'))  # 0: 개수, 1: 기간
            self.obj_StockChart.SetInputValue(2, date_to)  # 종료일
            self.obj_StockChart.SetInputValue(3, date_from)  # 시작일
            self.obj_StockChart.SetInputValue(4, 100)  # 요청 개수
            self.obj_StockChart.SetInputValue(5, list_field_key)  # 필드
            self.obj_StockChart.SetInputValue(6, ord('m'))  # 'D', 'W', 'M', 'm', 'T'
            self.obj_StockChart.BlockRequest()

            status = self.obj_StockChart.GetDibStatus()
            msg = self.obj_StockChart.GetDibMsg1()
            logger.debug("통신상태: {} {}".format(status, msg))
            if status != 0:
                return None

            cnt = self.obj_StockChart.GetHeaderValue(3)  # 수신개수
            for i in range(cnt):
                dict_item = {name: self.obj_StockChart.GetDataValue(pos, i) for pos, name in zip(range(len(list_field_name)), list_field_name)}
                for k, v in dict_item.items():
                    dict_chart[k].append(v)

            if not self.obj_StockChart.Continue:
                break
            self._wait()

        logger.debug("차트: {} {}".format(cnt, dict_chart))
        return pd.DataFrame(dict_chart, columns=list_field_name)

    def _wait(self):
        time_remained = self.obj_CpCybos.LimitRequestRemainTime
        cnt_remained = self.obj_CpCybos.GetLimitRemainCount(1)  # 0: 주문 관련, 1: 시세 요청 관련, 2: 실시간 요청 관련
        if cnt_remained <= 0:
            timeStart = time.time()
            while cnt_remained <= 0:
                time.sleep(time_remained / 1000)
                time_remained = self.obj_CpCybos.LimitRequestRemainTime
                cnt_remained = self.obj_CpCybos.GetLimitRemainCount(1)

In [13]:
creon = Creon()

In [5]:
result = creon.creon_7400_주식차트조회('233740', 20190905, 20190905)

In [6]:
len(result)

381

In [7]:
type(result)

pandas.core.frame.DataFrame

In [8]:
result.head(5)

,date,time,open,high,low,close,volume
0,20190905,1530,7240,7240,7240,7240,445439
1,20190905,1520,7225,7230,7215,7225,68706
2,20190905,1519,7220,7230,7220,7220,95360
3,20190905,1518,7220,7225,7215,7225,48369
4,20190905,1517,7220,7225,7215,7220,47957


In [34]:
def dump_date(creon, symbol, datestr):
    df = creon.creon_7400_주식차트조회(symbol, datestr, datestr)
    ofname = datestr + ".csv"
    df.to_csv(ofname)
    print("Wrote", ofname)

In [38]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)

In [35]:
start_date = datetime.date(2018, 1, 1)

In [36]:
end_date = datetime.date(2019, 8, 31)

In [ ]:
for date in daterange(start_date, end_date):
    if date.weekday() <= 4:  # Mon to Fri
        datestr = date.strftime("%Y%m%d")
        dump_date(creon, "251340", datestr)
        time.sleep(1)